In [2]:
import pandas as pd
import googlemaps
from itertools import tee
import urllib.request
import json
import re

In [138]:
df = pd.read_csv('C:/Users/lonsc47/Documents/Test.csv')
df.head(3)

,Unnamed: 0,Name,Department,Postcode Start,Postcode End
0,0,Test A,Dep A,TN161DP,SE14YJ
1,1,Test B,Dep B,E11GU,SE14YJ
2,2,Test C,Dep C,B25NY,SE14YJ


In [148]:
destination = "SE1%204YJ"
pub_list_distances = []
pub_list_times_hours = []
pub_list_times_mins = []
car_list_distances = []
car_list_times_hours = []
car_list_times_mins = []

#Get public transport distances / times for each row one by one - feed them into api using url#
for num, row in enumerate(df['Postcode Start']):
    #substring the postcode to get first 3-4 chars and last 3 chars separately#
    start_pos_code = df['Postcode Start'][num][:-3:]
    end_pos_code = df['Postcode Start'][num][-3::]
    origin = start_pos_code+"%20"+end_pos_code
    
    #write api url with origin, destination, mode and api key - generates a json#
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins="+origin+"&destinations="+destination+"&mode=transit&key=[KEY HERE]"
    res = urllib.request.urlopen(url).read()
    data = json.loads(res.decode())
    
    #grab distance and time from json#
    distance = data["rows"][0]["elements"][0]["distance"]["text"]
    time = data["rows"][0]["elements"][0]["duration"]["text"]
    
    #append distance and time to lists - will be added to final dataframe#
    pub_list_distances.append(distance[:-3])
    pub_list_times_hours.append(time)
    
    #use regex to extract hour and minutes separately to create a total minutes value#
    mins = re.search('[0-9]{1,}(?= [m])',time).group(0)
    try:
        hours = re.search('(?<! )[0-9]{1,}(?= [h])',time).group(0)
    except:
        hours = 0
        pass
    total_mins = int(mins) + (int(hours)*60)
    pub_list_times_mins.append(total_mins)

    
    
#Get driving distances / times#
for num, row in enumerate(df['Postcode Start']):
    start_pos_code = df['Postcode Start'][num][:-3:]
    end_pos_code = df['Postcode Start'][num][-3::]
    origin = start_pos_code+"%20"+end_pos_code
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins="+origin+"&destinations="+destination+"&key=[KEY HERE]"
    res = urllib.request.urlopen(url).read()
    data = json.loads(res.decode())
    distance = data["rows"][0]["elements"][0]["distance"]["text"]
    time = data["rows"][0]["elements"][0]["duration"]["text"]
    car_list_distances.append(distance[:-3])
    car_list_times_hours.append(time)
    mins = re.search('[0-9]{1,}(?= [m])',time).group(0)
    try:
        hours = re.search('(?<! )[0-9]{1,}(?= [h])',time).group(0)
    except:
        hours = 0
        pass
    total_mins = int(mins) + (int(hours)*60)
    car_list_times_mins.append(total_mins)
    
print('done')

In [149]:
#Set lists as columns in df#
df['Distance by Public Transport (miles)'] = pub_list_distances
df['Time by Public Transport'] = pub_list_times_hours
df['Time by Public Transport (mins)'] = pub_list_times_mins
df['Distance by Car (miles)'] = car_list_distances
df['Time by Car'] = car_list_times_hours
df['Time by Car (mins)'] = car_list_times_mins
df.head(1)

,Unnamed: 0,Name,Department,Postcode Start,Postcode End,Distance by Public Transport (miles),Time by Public Transport,Time by Public Transport (mins),Distance by Car (miles),Time by Car,Time by Car (mins)
0,0,Test A,Dep A,TN161DP,SE14YJ,22.8,1 hour 25 mins,85,28.5,1 hour 6 mins,66


In [150]:
#Write result#
df.to_csv('C:/Users/lonsc47/Documents/Test.csv')